# モデルの訓練と保存
Google Colaboratoryで画像識別用のモデルを構築し、訓練します。  
今回は、cifar-10を訓練データに使います。  
訓練済みのモデルは保存し、ダウンロードします。

## バージョンの修正と確認
docker内部のkerasとtensorflowの環境に合わせる。そうしないと、google colabで生成したモデルがdocker内部の環境に適用できなくなってしまう。

In [ ]:
!pip list | grep Keras

Keras-Preprocessing           1.1.2


In [ ]:
!pip uninstall keras

Found existing installation: keras 2.8.0
Uninstalling keras-2.8.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? y
  Successfully uninstalled keras-2.8.0


In [ ]:
!pip install keras==2.3.1

     |████████████████████████████████| 377 kB 5.1 MB/s 
     |████████████████████████████████| 50 kB 9.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.3.1 which is incompatible.


In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.8.0


In [ ]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3 MB 36 kB/s 
     |████████████████████████████████| 3.8 MB 46.3 MB/s 
     |████████████████████████████████| 449 kB 52.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=9b0208452b16f3fccde00399c454894f1654dbe3c29783e45d9489010f71061f
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
ERROR: pip's dependency res

In [ ]:
import tensorflow
import keras
print(tensorflow.__version__)
print(keras.__version__)

2.0.0
2.3.1


Using TensorFlow backend.


## 訓練データの読み込み
kerasで既に用意されているデータセットであるcifar-10を読み込みます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

データが問題なく取得できているかを確認します。

In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
t_train.shape

(50000, 1)

In [ ]:
x_test.shape

(10000, 32, 32, 3)

In [ ]:
t_test.shape

(10000, 1)

## モデルの構築
CNNのモデルを構築します。

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam 

batch_size = 32
epochs = 12
n_class = 10
# one-hot表現に変換します。
t_train = to_categorical(t_train, n_class)  
t_test = to_categorical(t_test, n_class)
#簡単なモデルを構築します。
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))  # ゼロパディング、バッチサイズ以外の画像の形状を指定
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 一次元の配列に変換
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # ドロップアウト
model.add(Dense(n_class))
model.add(Activation("softmax"))

model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 15, 15, 64)       

## 学習
モデルを訓練します。  
ImageDataGeneratorを使用する際は、学習にfit_generatorメソッドを使います。  
学習には時間がかかりますので、編集→ノートブックの設定のハードウェアアクセラレーターでGPUを選択しましょう。

In [ ]:
x_train = x_train / 255  # 0から1の範囲に収める
x_test = x_test / 255
#データの水増しを行っています。(回転・平行移動・拡大など)
generator = ImageDataGenerator(
           rotation_range=0.2,
           width_shift_range=0.2,
           height_shift_range=0.2,
           shear_range=10,
           zoom_range=0.2,
           horizontal_flip=True)
generator.fit(x_train)
#学習経過をログとして残しています。
#引数としては、学習データに加えて、バッチサイズ、学習回数、検証データを渡しています。
history = model.fit_generator(generator.flow(x_train, t_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=(x_test, t_test))

Epoch 1/12
1563/1563 [==============================] - 230s 147ms/step - loss: 1.7897 - accuracy: 0.3326 - val_loss: 1.4886 - val_accuracy: 0.4411
Epoch 2/12
1312/1563 [========================>.....] - ETA: 35s - loss: 1.4980 - accuracy: 0.4542

KeyboardInterrupt: ignored

## 学習の推移
学習の推移を表示します。  

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history["loss"]  # 訓練用データの誤差
train_acc = history.history["accuracy"]  # 訓練用データの精度
val_loss = history.history["val_loss"]  # 検証用データの誤差
val_acc = history.history["val_accuracy"]  # 検証用データの精度

plt.plot(np.arange(len(train_loss)), train_loss, label="loss")
plt.plot(np.arange(len(val_loss)), val_loss, label="val_loss")
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc)), train_acc, label="accuracy")
plt.plot(np.arange(len(val_acc)), val_acc, label="val_acc")
plt.legend()
plt.show()

## 評価
モデルの評価を行います。  

In [ ]:
loss, accuracy = model.evaluate(x_test, t_test)
print(loss, accuracy)

313/313 [==============================] - 1s 4ms/step - loss: 0.8266 - accuracy: 0.7192
0.8265902400016785 0.7192000150680542


## モデルの保存とダウンロード
他の環境でも使えるように、モデルを保存しダウンロードします。

In [ ]:
from google.colab import files

model.save("image_classifier.h5",  include_optimizer=False)
files.download("image_classifier.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

.h5の拡張子を持つモデルのファイルが、ローカルに保存されました。